In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]

,pickup_datetime,pickup_location_id
1000,2023-01-01 00:45:01,231
1001,2023-01-01 00:26:54,238
1002,2023-01-01 00:44:19,43
1003,2023-01-01 00:21:19,239
1004,2023-01-01 00:43:50,151
1005,2023-01-01 00:25:32,132
1006,2023-01-01 00:11:14,237
1007,2023-01-01 00:29:35,163
1008,2023-01-01 00:47:34,114
1009,2023-01-01 00:11:52,141


In [6]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2993140 entries, 0 to 2993139
Data columns (total 2 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_datetime     datetime64[us]
 1   pickup_location_id  int64         
dtypes: datetime64[us](1), int64(1)
memory usage: 45.7 MB


In [7]:
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor('h')
rides.tail()

,pickup_datetime,pickup_location_id,pickup_hour
2993135,2023-01-31 23:58:34,107,2023-01-31 23:00:00
2993136,2023-01-31 23:31:09,112,2023-01-31 23:00:00
2993137,2023-01-31 23:01:05,114,2023-01-31 23:00:00
2993138,2023-01-31 23:40:00,230,2023-01-31 23:00:00
2993139,2023-01-31 23:07:32,262,2023-01-31 23:00:00


In [8]:
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.tail()

,pickup_hour,pickup_location_id,rides
69772,2023-01-31 23:00:00,249,137
69773,2023-01-31 23:00:00,256,2
69774,2023-01-31 23:00:00,261,5
69775,2023-01-31 23:00:00,262,11
69776,2023-01-31 23:00:00,263,41


In [9]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [10]:
hour_col = "pickup_hour"
location_col = "pickup_location_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

In [11]:
agg_data_filled.head(15)

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
5,2023-01-01 05:00:00,2,0
6,2023-01-01 06:00:00,2,0
7,2023-01-01 07:00:00,2,0
8,2023-01-01 08:00:00,2,0
9,2023-01-01 09:00:00,2,0


In [12]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none"
    )

    fig.show()

In [13]:
plot_rides(agg_data_filled, locations=[42, 43])

In [14]:
path = Path('..') / 'data' / 'processed' / f'ts_data_{year}_{month:02}.parquet'
agg_data_filled.to_parquet(path, engine="pyarrow", index=False)